In [8]:
pip install sentence-transformers


     -------------------------------------- 275.9/275.9 kB 5.7 MB/s eta 0:00:00
     -------------------------------------- 468.0/468.0 kB 7.4 MB/s eta 0:00:00
     --------------------------------------- 10.0/10.0 MB 10.5 MB/s eta 0:00:00
     -------------------------------------- 308.9/308.9 kB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.3
    Uninstalling safetensors-0.3.3:
      Successfully uninstalled safetensors-0.3.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installatio

In [9]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

c:\Users\prakash.pandey\Miniconda3\envs\mle-dev\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
data = pd.read_csv('./akc-data-latest.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Dog_name                     277 non-null    object 
 1   description                  277 non-null    object 
 2   temperament                  276 non-null    object 
 3   popularity                   198 non-null    object 
 4   min_height                   277 non-null    float64
 5   max_height                   277 non-null    float64
 6   min_weight                   275 non-null    float64
 7   max_weight                   275 non-null    float64
 8   min_expectancy               274 non-null    float64
 9   max_expectancy               274 non-null    float64
 10  group                        277 non-null    object 
 11  grooming_frequency_value     270 non-null    float64
 12  grooming_frequency_category  270 non-null    object 
 13  shedding_value      

In [4]:
data.Dog_name.nunique()

277

In [5]:
data.describe()

,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value
count,277.000000,277.000000,275.000000,275.000000,274.000000,274.000000,270.000000,257.000000,271.000000,253.000000,252.000000
mean,44.225801,52.720588,17.888858,27.291416,11.306569,13.832117,0.425926,0.529183,0.712915,0.624506,0.620635
std,14.238298,15.885454,12.290600,19.061416,1.817949,2.016668,0.198306,0.189068,0.168927,0.247271,0.201713
min,12.700000,17.780000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000
25%,33.020000,38.100000,8.164663,12.927383,10.000000,13.000000,0.200000,0.400000,0.600000,0.400000,0.400000
50%,45.085000,53.340000,15.875733,24.947580,12.000000,14.000000,0.400000,0.600000,0.600000,0.600000,0.600000
75%,55.880000,66.040000,22.679619,34.019428,12.000000,15.000000,0.600000,0.600000,0.800000,0.800000,0.800000
max,76.200000,88.900000,68.038855,108.862169,16.000000,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
data.columns

Index(['Dog_name', 'description', 'temperament', 'popularity', 'min_height',
       'max_height', 'min_weight', 'max_weight', 'min_expectancy',
       'max_expectancy', 'group', 'grooming_frequency_value',
       'grooming_frequency_category', 'shedding_value', 'shedding_category',
       'energy_level_value', 'energy_level_category', 'trainability_value',
       'trainability_category', 'demeanor_value', 'demeanor_category'],
      dtype='object')

In [12]:
data.dropna(axis=0, inplace=True)

In [13]:
data.shape

(188, 21)

In [14]:
def create_breed_text(row: pd.Series) -> str:
    """
    Create an aggregated text representation for each breed using various NLP columns.
    Exclude 'Dog_name' from the aggregation (as it is the target) or include it if it helps provide context.
    Here, we combine multiple descriptive fields to form a richer document for each breed.
    """
    # You can adjust the columns you want to include. Here, we combine several key descriptive fields.
    columns_to_include = [
        'description',
        'temperament',
        'energy_level_category',
        'trainability_category',
        'demeanor_category',
        'grooming_frequency_category',
        'shedding_category'
    ]
    # Concatenate available text fields, filtering out NaNs.
    texts = [str(row[col]) for col in columns_to_include if pd.notna(row[col])]
    return " ".join(texts)

def create_breed_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> torch.Tensor:
    """
    For each breed, create a text string that combines multiple NLP columns.
    Then encode all these aggregated texts into embeddings.
    """
    # Create a new column with aggregated text for each breed.
    df['breed_text'] = df.apply(create_breed_text, axis=1)
    texts = df['breed_text'].tolist()
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings

In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = create_breed_embeddings(data, model=model)

c:\Users\prakash.pandey\Miniconda3\envs\mle-dev\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prakash.pandey\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
embeddings

tensor([[-0.0575,  0.0033,  0.0428,  ..., -0.0549,  0.0493,  0.0666],
        [-0.0670,  0.0820,  0.0762,  ..., -0.0179,  0.0767, -0.0050],
        [-0.0092,  0.0266,  0.0282,  ..., -0.0150, -0.0589,  0.0975],
        ...,
        [ 0.0129,  0.0251,  0.0221,  ...,  0.0050,  0.0036,  0.0264],
        [ 0.0508,  0.0530,  0.0181,  ..., -0.0454,  0.0456,  0.0269],
        [ 0.0538, -0.0459,  0.0782,  ..., -0.0160,  0.0280,  0.0713]])

In [19]:
len(embeddings), len(embeddings[0])

(188, 384)

In [20]:
def get_recommendation(query: str, df: pd.DataFrame, breed_embeddings: torch.Tensor, model: SentenceTransformer, top_k: int = 1) -> pd.DataFrame:
    """
    Compute the embedding for the user query and compare it to the breed embeddings.
    Return the top_k breeds ranked by cosine similarity.
    """
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_embedding, breed_embeddings)[0]

    top_results = torch.topk(cosine_scores, k=top_k)
    recommended_breeds = df.iloc[top_results.indices.cpu().numpy()].copy()
    recommended_breeds['similarity_score'] = top_results.values.cpu().numpy()
    return recommended_breeds

In [26]:
query = "I have young kids and limited time for grooming. Which breed would suit my family?"
recommendation = get_recommendation(query, data, embeddings, model, top_k=3)

In [27]:
recommendation[['Dog_name',
        'description',
        'temperament',
        'energy_level_category',
        'trainability_category',
        'demeanor_category',
        'grooming_frequency_category',
        'shedding_category'
    ]]

,Dog_name,description,temperament,energy_level_category,trainability_category,demeanor_category,grooming_frequency_category,shedding_category
82,Collie,The Collie is a large but lithe herder standin...,"Devoted, Graceful, Proud",Regular Exercise,Easy Training,Friendly,2-3 Times a Week Brushing,Seasonal
34,Bergamasco Sheepdog,"Beneath the one-of-a-kind coat is a large, mus...","Independent, Sociable, Intelligent",Calm,Agreeable,Reserved with Strangers,Occasional Bath/Brush,Infrequent
109,Flat-Coated Retriever,The Flat-Coated Retriever’s eponymous flat-lyi...,"Cheerful, Optimistic, Good-Humored",Needs Lots of Activity,Eager to Please,Friendly,Weekly Brushing,Regularly


In [28]:
query = "What breeds are known for being both protective and good with families?"
recommendation = get_recommendation(query, data, embeddings, model, top_k=3)
recommendation[['Dog_name',
        'description',
        'temperament',
        'energy_level_category',
        'trainability_category',
        'demeanor_category',
        'grooming_frequency_category',
        'shedding_category'
    ]]

,Dog_name,description,temperament,energy_level_category,trainability_category,demeanor_category,grooming_frequency_category,shedding_category
88,Dalmatian,"The Dalmatian’s delightful, eye-catching spots...","Dignified, Smart, Outgoing",Energetic,Agreeable,Alert/Responsive,Weekly Brushing,Frequent
106,Field Spaniel,Field Spaniels bear a family resemblance to Co...,"Sweet, Fun-Loving, Sensitive",Regular Exercise,Agreeable,Reserved with Strangers,Weekly Brushing,Regularly
34,Bergamasco Sheepdog,"Beneath the one-of-a-kind coat is a large, mus...","Independent, Sociable, Intelligent",Calm,Agreeable,Reserved with Strangers,Occasional Bath/Brush,Infrequent


In [30]:
recommendation.description.iloc[0]

'The Dalmatian’s delightful, eye-catching spots of black or liver adorn one of the most distinctive coats in the animal kingdom. Beneath the spots is a graceful, elegantly proportioned trotting dog standing between 19 and 23 inches at the shoulder. Dals are muscular, built to go the distance; the powerful hindquarters provide the drive behind the smooth, effortless gait.The Dal was originally bred to guard horses and coaches, and some of the old protective instinct remains. Reserved and dignified, Dals can be aloof with strangers and are dependable watchdogs. With their preferred humans, Dals are bright, loyal, and loving house dogs. They are strong, active athletes with great stamina—a wonderful partner for runners and hikers. The dignified Dalmatian, dogdom\'s citizen of the world, is famed for his spotted coat and unique job description. During their long history, these "coach dogs" have accompanied the horse-drawn rigs of nobles, gypsies, and firefighters.'